# Assignment 1: Basics

In [3]:
import os

## 1 - Assignment Overview

## 2 - Byte-Pair Encoding Tokenizer

### 2.1 - The Unicode standard

In [90]:
chr0 = chr(0)

In [91]:
print(f'Direct print: "{chr0}"; representation: "{chr0.__repr__()}"')

Direct print: " "; representation: "'\x00'"


(a) It represents the NULL character

(b) Printing it gives nothing, but it's representation is \x00, which is a bytes literal for 0, which represents U+0000 in Unicode

(c) Printing shows nothing, but the representation is still the \x00

### 2.2 - Unicode Encodings


In [92]:
test_string = "hello! こんにちは!"

In [93]:
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'


In [94]:
print(type(utf8_encoded))

<class 'bytes'>


In [95]:
list (utf8_encoded)

[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

In [96]:
print(f'length of test string: {len(test_string)}, vs length of utf encoding {len(utf8_encoded)}')

length of test string: 13, vs length of utf encoding 23


In [97]:
for id in range(len(utf8_encoded)):
    try:
        print(f'first {id} utf8 characters: "{utf8_encoded[0:id].decode("utf-8")}"')
    except:
        print(f'first {id} utf8 characters: <<decoding failed>>')

first 0 utf8 characters: ""
first 1 utf8 characters: "h"
first 2 utf8 characters: "he"
first 3 utf8 characters: "hel"
first 4 utf8 characters: "hell"
first 5 utf8 characters: "hello"
first 6 utf8 characters: "hello!"
first 7 utf8 characters: "hello! "
first 8 utf8 characters: <<decoding failed>>
first 9 utf8 characters: <<decoding failed>>
first 10 utf8 characters: "hello! こ"
first 11 utf8 characters: <<decoding failed>>
first 12 utf8 characters: <<decoding failed>>
first 13 utf8 characters: "hello! こん"
first 14 utf8 characters: <<decoding failed>>
first 15 utf8 characters: <<decoding failed>>
first 16 utf8 characters: "hello! こんに"
first 17 utf8 characters: <<decoding failed>>
first 18 utf8 characters: <<decoding failed>>
first 19 utf8 characters: "hello! こんにち"
first 20 utf8 characters: <<decoding failed>>
first 21 utf8 characters: <<decoding failed>>
first 22 utf8 characters: "hello! こんにちは"


In [98]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    try:
        output_str = "".join([bytes([b]).decode("utf-8") for b in bytestring])
    except:
        output_str = "ERROR"
    return output_str

decode_utf8_bytes_to_str_wrong("hello!".encode("utf-8"))
decode_utf8_bytes_to_str_wrong("hello! こんにちは!".encode("utf-8"))

'ERROR'

In [99]:
bytes_list = list("こ".encode("utf-8"))
print(f'For character "こ", this is the bytes list: {list("こ".encode("utf-8"))}, representing {"こ".encode("utf-8")}')
print(f'If I decode all 3 bytes together I get "{bytes(bytes_list).decode("utf-8")}"')
print(f'If I decode only first 2 bytes I get "{bytes(bytes_list[0:2]).decode("utf-8")}"')

For character "こ", this is the bytes list: [227, 129, 147], representing b'\xe3\x81\x93'
If I decode all 3 bytes together I get "こ"


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data

(a) Reasons to prefer UTF-8 encoded bytes: Bytes are naturally well-aligned with the infrastructure of the internet and codes. UTF-16 and UTF-32 are multi-byte, introducing complexity

(b) `decode_utf8_bytes_to_str_wrong` doesn't work for characters that are longer than 1 byte.

(c) `\xe3\x81`, which is only the first 2 of 3 bytes in こ (ko)

### 2.3 - Subword Tokenization

### 2.4 - BPE Tokenizer Training

In [14]:
text_to_tokenize = "Some text that I'll pre-tokenize. こんにちは!"

In [15]:
import regex as re

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

re.findall(PAT, text_to_tokenize)

['Some',
 ' text',
 ' that',
 ' I',
 "'ll",
 ' pre',
 '-',
 'tokenize',
 '.',
 ' こんにちは',
 '!']

In [16]:
PAT_BYTE = rb"'(?:[sdmt]|ll|ve|re)| ?[A-Za-z]+| ?[0-9]+| ?[^\sA-Za-z0-9]+|\s+(?!\S)|\s+"

re.findall(PAT_BYTE, text_to_tokenize.encode("utf_8"))

[b'Some',
 b' text',
 b' that',
 b' I',
 b"'ll",
 b' pre',
 b'-',
 b'tokenize',
 b'.',
 b' \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!']

In [ ]:
re.findall(PAT,"hello! こんにちは!")

['hello', '!', ' こんにちは', '!']

Algorithm 1 of Sennrich et al. [2016]:

In [102]:
import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i],symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [18]:

vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2,'n e w e s t </w>':6, 'w i d e s t </w>':3}

num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(f'iteration {i}: best match "{best}"')

print(vocab)

NameError: name 'get_stats' is not defined

Relation between `num_merges` and `vocab` size:

- If you run a low number of merges, you're ending up with almost the same encoding as UTF-8

- The more you run the, closer you get to full word encoding, where each word is represented directly by a token.

- However, if you introduce new words, they will obviously still be encoded using existing pieces, that are smaller than the word

### 2.5 - Experimenting with BPE Tokenizer Training

In [4]:
with open("data/owt_valid.txt", "rb") as file:    
    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    print(f'file size: {file_size}')
    file.seek(0)

    desired_num_chunks = 3
    chunk_size = file_size // desired_num_chunks
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

file size: 289998753


In [128]:
!stat -f %z data/owt_valid.txt

6896.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


289998753


Diary on profiling

1. I ran profiling on TinyStoriesV2-GPT4-train.txt and it gave me the entries for profiles/profile_with_pretokenization. Time spent is roughly split by build_pretokens, get_stats and apply_merge
2. We can speed up build_pretokens by running it in parallel
3. We can speed up get_stats and apply_merge by using a better algorithm as explained at the bottom of page 8 in [cs336_spring2025_assignment1_basics.pdf](cs336_spring2025_assignment1_basics.pdf)
4. We should focus first on improving the algorithm
5. I implemented a completely new algorithm that uses minheaps that contain the occurences of affected pairs. As you can see in `output_owt_train_20251104_212059.log` this gave a total execution time of 13 minutes for computing pretokens but about 4 hours for the merging (21:37-01:34)
6. I then ran this through Scalene and it seems that most gains were to be made in a single line of code: `negc, _seq_left, _seq_right, stamp, pair = heapq.heappop(heap)`.
7. It turns out that the comparison logic is probably expensive, as it is not done in the native C implementation of `heapq` but in a custom Python class `class _DescendingSeq` which I introduced to ensure lexicographical maximum ordering. This could perhaps save up to 19% of total execution time, so we could explore this to shave off something like 45 minutes perhaps.
8. I decided to only remove `isinstance` checks, and not focus on the rest.




BPE on TinyStories:

(a) Overview 
- Use `output_TinyStoriesV2-GPT4-train_20251104_211758.log` for reference
- pretokenization takes 0:02:37 (21:17:58-21:20:35)
- merging takes very little time 0:00:08 (21:20:36-21:20:44)
- total takes under 3 minutes

(b) Profile
- Use `profiles/profile_TinyStoriesV2-GPT4-train_20251105_092150.html` 
- Note that most optimizable time (indicated with `python`) is spent in `negc, _seq_left, _seq_right, stamp, pair = heapq.heappop(heap)`
- However, the reality is that this is the `_DescendingSeq` custom class that is used for sorting, as discussed above

BPE on Open Web Text

In [ ]:
import pickle
from pathlib import Path

def load_latest_vocab(pattern: str):
    matches = sorted(Path("outputs").glob(pattern))
    if not matches:
        raise FileNotFoundError(f"No files matched pattern: {pattern}")
    vocab_path = matches[-1]
    with open(vocab_path, "rb") as fh:
        vocab = pickle.load(fh)
    return vocab_path, vocab

owt_path, owt_vocab = load_latest_vocab("output_owt_train_*_vocab.pkl")
ts_path, ts_vocab = load_latest_vocab("output_TinyStoriesV2-GPT4-train_*_vocab.pkl")

print(f"Loaded OWT vocab ({len(owt_vocab):,} entries) from {owt_path.name}")
print(f"Loaded TinyStories vocab ({len(ts_vocab):,} entries) from {ts_path.name}")


Loaded OWT vocab (32,000 entries) from output_owt_train_20251105_204523_vocab.pkl
Loaded TinyStories vocab (10,000 entries) from output_TinyStoriesV2-GPT4-train_20251105_092150_vocab.pkl


In [16]:
# Get the 5 keys with the longest values in owt_vocab
longest_items = sorted(owt_vocab.items(), key=lambda kv: len(kv[1]), reverse=True)[:20]
for k, v in longest_items:
    print(f"Key: {k}, Length: {len(v)}, Value: {v}")

Key: 25810, Length: 64, Value: b'\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82'
Key: 25824, Length: 64, Value: b'----------------------------------------------------------------'
Key: 31257, Length: 48, Value: b'\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94'
Key: 10903, Length: 32, Value: b'--------------------------------'
Key: 15944, Length: 32, Value: b'________________________________'
Key: 16882, Length: 32, Value: b'\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82'
Key: 25134, Length: 32, Value: b'================================'
Key: 28569, Length:


(a) Overview
* Use `outputs/output_owt_train_20251105_204523.log` for reference, though I turned off debug so the amount of information is less
* For profile please see `profiles/profile_owt_train_20251105_204523.html` which shows interestingly that the situation is better than for TinyStories: Proportionally less time is spent on `python` (18% only).
* The computation does, however, halt at various times for an extended amount of times (~minutes easily), which I think might be due to python memory cleanup or something related. That would ipmly that this system would run faster on a less resource-constrained system.
* Currently the maximum memory footprint is about 24GB, and the total runtime is under 2h. Note that the log above takes a precreated pretoken list and saves off about 20 minutes leading to a merge time of 1:32:05 (20:45:27-22:17:32).
* The longest 20 tokens contain some that just have high enough frequency, but also a file separator, garbage and other stuff. This is how chatGPT classifies them:

| Key(s) | Token Value (decoded) | Category | Explanation |
|---------|-----------------------|-----------|--------------|
| 25810, 16882 | ÃÂÃÂÃÂÃÂ... | **1. Encoding noise** | Double-encoded UTF-8 garbage from misinterpreted Unicode (“ÃÂ” artifacts). |
| 25824, 10903, 15944, 25134, 28569, 31145 | ----------------, --------------------------------, ________________________________, ================================, ................................, ******************************** | **2. Visual separators (ASCII)** | Markdown or ASCII-art dividers made of repeated punctuation. |
| 31257, 15278 | ————————————————, ———————— | **3. Unicode punctuation** | Repeated em dashes (`\xe2\x80\x94`) used as stylistic or section breaks. |
| 23318, 24257, 28259, 14284, 16281, 25686, 26061, 27022, 28477 | disproportionately, telecommunications, environmentalists, responsibilities, unconstitutional, cryptocurrencies, disproportionate, misunderstanding, counterterrorism | **4. Common long words** | Whole high-frequency English words learned as single tokens. |
| 260 | <\|file_separator\|> | **5. Special marker** | Control token marking file boundaries during dataset creation. |


In [6]:
def vocab_values(vocab_dict):
    return set(vocab_dict.values())

owt_tokens = vocab_values(owt_vocab)
ts_tokens = vocab_values(ts_vocab)

owt_only_tokens = owt_tokens - ts_tokens
ts_only_tokens = ts_tokens - owt_tokens
overlap_tokens = owt_tokens & ts_tokens

def describe(label, diff_set, base_size, show_sample=25):
    percentage = (len(diff_set) / base_size * 100) if base_size else 0.0
    print(f"{label}: {len(diff_set):,} tokens ({percentage:.4f}% of its vocab)")
    if diff_set:
        preview = sorted(diff_set, key=lambda tok: (-len(tok), tok))[:show_sample]
        print("  sample:")
        for tok in preview:
            print(f"   {tok}")

describe("OWT-only", owt_only_tokens, len(owt_tokens))
describe("TinyStories-only", ts_only_tokens, len(ts_tokens))
print(f"Shared tokens: {len(overlap_tokens):,} (covers {len(overlap_tokens) / len(ts_tokens) * 100:.4f}% of TinyStories)")


OWT-only: 24,678 tokens (77.1188% of its vocab)
  sample:
   b'----------------------------------------------------------------'
   b'\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82'
   b'\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94'
   b'********************************'
   b'--------------------------------'
   b'................................'
   b'================================'
   b'________________________________'
   b'\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82'
   b'\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\xe2\x80\x94\x

(b) Comparison and contrast between OWT and TinyStories:

* OWT Contains a lot more decoding artifacts and coding artifacts
* TinyStories contain more long words
* These differences could be due to the different classes of documents int he corpora, which seem 